执行操作： abs_energy/这一段总的样本数

In [2]:
import pandas as pd
import numpy as np
import joblib
from joblib import Parallel,delayed
import os

In [2]:
APPLY_COLS = ['abs_energy', 'absolute_sum_of_changes', 'longest_strike_below_mean', 
'longest_strike_above_mean', 'count_above_mean', 'count_below_mean', 
'sum_of_reoccurring_values', 'sum_of_reoccurring_data_points']

In [3]:
concat = joblib.load('./concats/concat_01.lz4')
concat.shape

(110027, 37)

In [4]:
concat.columns

Index(['RULR', 'csv_no', 'CL', 'CLI', 'spindle_load', 'Current_1__abs_energy',
       'Current_1__kurtosis', 'Current_1__length', 'Current_1__mean',
       'Current_1__mean_abs_change', 'Current_1__mean_change',
       'Current_1__skewness', 'Current_1__variance', 'Vibration_1__abs_energy',
       'Vibration_1__kurtosis', 'Vibration_1__length', 'Vibration_1__mean',
       'Vibration_1__mean_abs_change', 'Vibration_1__mean_change',
       'Vibration_1__skewness', 'Vibration_1__variance',
       'Vibration_2__abs_energy', 'Vibration_2__kurtosis',
       'Vibration_2__length', 'Vibration_2__mean',
       'Vibration_2__mean_abs_change', 'Vibration_2__mean_change',
       'Vibration_2__skewness', 'Vibration_2__variance',
       'Vibration_3__abs_energy', 'Vibration_3__kurtosis',
       'Vibration_3__length', 'Vibration_3__mean',
       'Vibration_3__mean_abs_change', 'Vibration_3__mean_change',
       'Vibration_3__skewness', 'Vibration_3__variance'],
      dtype='object')

In [3]:
def scale_tsfeature(data_no, apply_cols=None):
    if apply_cols==None:
        apply_cols = []
    # scale abs_energy
    concat = joblib.load('./concats/concat_0%d.lz4'%data_no)
    # scale
    length_arr = concat['Current_1__length'].values
    
    def func(filter_col, apply_cols):
        for col in apply_cols:
            if col in filter_col:
                return True
        return False
    
    scale_cols = [col for col in concat.columns if func(col, apply_cols)]
    for col in scale_cols:
        concat[col+'_scale'] = concat[col]/length_arr
    # drop useless cols
    drop_cols = scale_cols + [col for col in concat.columns if 'length' in col]
    return concat

# def calc_moving_speed(df):
#     diff_df = df[['x','y','z','distance_xy','distance_xz','distance_yz','distance_xyz', 'CL']].copy().diff()
#     speed_arr = diff_df[['x','y','z','distance_xy','distance_xz','distance_yz','distance_xyz']].values / diff_df[['CL']].values
#     speed_cols = [col+'__speed' for col in ['x','y','z','distance_xy','distance_xz','distance_yz','distance_xyz']]
#     speed_df = pd.DataFrame(data=speed_arr, columns=speed_cols)
#     result = pd.concat([df, speed_df], axis=1)
    
#     #　剔除nans和infs
#     cond = np.isfinite(result).all(axis=1)
#     print('nans和infs的数量：', result.shape[0]-cond.sum())
#     result = result[cond]
    
#     #　剔除重复值
#     result.drop_duplicates(subset='CL', inplace=True)
    
#     result.reset_index(drop=True, inplace=True)
#     return result

def write_files(data_no):
    df_1 = scale_tsfeature(data_no, APPLY_COLS)
#     df_2 = calc_moving_speed(df_1)
    df_2 =df_1
    if not os.path.exists('./concats_more'):
        os.mkdir('./concats_more')
    joblib.dump(df_2, './concats_more/concat_0%d.lz4'%data_no, compress='lz4')

In [4]:
write_files(1)
write_files(2)
write_files(3)